In [55]:
from database.strategy import Strategy
from database.market import Market
from database.sec import SEC
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [56]:
strat_db = Strategy("recs")
market = Market()

In [57]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [58]:
strat_db.connect()
sim = strat_db.retrieve_data("sim")
strat_db.close()

In [59]:
strat_db.connect()
recs = strat.retrieve_data("recs")
strat_db.close()

In [60]:
sim["week"] = [x.week for x in sim["date"]]
sim["year"] = [x.year for x in sim["date"]]

In [61]:
recs["week"] = [x.week for x in recs["date"]]
recs["year"] = [x.year for x in recs["date"]]

In [62]:
wgs = sim.groupby(["year","week","ticker"]).mean().reset_index()

In [64]:
ticker_categories_weeklys = []
for ticker in wgs["ticker"].unique():
    ticker_wgs = wgs[wgs["ticker"] == ticker]
    ticker_wgs["actualclose"] = (ticker_wgs["adjclose"].shift(-1))
    ticker_wgs = ticker_wgs[:-1]
    ticker_wgs["actual_delta"] = (ticker_wgs["actualclose"] - ticker_wgs["adjclose"]) / ticker_wgs["adjclose"]
    ticker_wgs["sync"] = [0 if row[1]["actual_delta"] * row[1]["weekly_price_regression_prediction_delta"] >= 0 else 1 for row in ticker_wgs.iterrows()]
    ticker_wgs["category"] = ticker_wgs["sync"].rolling(window=5).mean()
    ticker_wgs = ticker_wgs[5:]
    ticker_wgs["category"] = [round(x) for x in ticker_wgs["category"]]
    ticker_categories_weeklys.append(ticker_wgs)
tcw = pd.concat(ticker_categories_weeklys)

<ipython-input-64-2f40673f4a7a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_wgs["actualclose"] = (ticker_wgs["adjclose"].shift(-1))


In [93]:
final_sim = recs.merge(tcw[tcw["week"] == tcw["week"].max()][["year","week","ticker","category"]],on="ticker",how="left").dropna()
final_sim["predicted_delta"] = [-1 * row[1]["weekly_price_regression_prediction_delta"] \
                          if row[1]["category"] == 1 else row[1]["weekly_price_regression_prediction_delta"] \
                          for row in final_sim.iterrows()]

In [94]:
len(recs["ticker"].unique())

424

In [96]:
final_sim[(final_sim["trade_signal_prediction"] == 1) & (final_sim["trade_signal_prediction_score"] >= 0.7)] \
                .sort_values("predicted_delta",ascending=False).head(10)[["ticker","category","predicted_delta"]]

,ticker,category,predicted_delta
177,GE,0,7.118398
309,PBCT,0,4.414631
24,AAL,0,3.916141
229,KEY,0,3.605455
186,HBI,0,2.523514
200,HPQ,0,2.239716
72,COG,0,2.147529
399,UNM,0,2.094075
293,NLOK,0,1.972331
420,WU,0,1.819453
